In [2]:
import pandas as pd
from connect_to_api import ResRobot
import requests
import streamlit as st
from dotenv import load_dotenv
import os
from datetime import datetime

# Load environment variables
load_dotenv()

API_KEY = os.getenv("API_KEY")

timetable = ResRobot()

In [4]:
location_name = "Angered Centrum"
ext_id = timetable.get_location_id(location_name)

if isinstance(ext_id, int):  
    ext_id = str(ext_id)  
else:
    raise ValueError(f"No valid extId found for '{location_name}'. Check the stop name and try again.")

In [5]:
# Function to fetch timetable data
def fetch_timetable(ext_id):
    url = f"https://api.resrobot.se/v2.1/departureBoard?id={ext_id}&format=json&accessId={API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if "Departure" not in data:
            st.error("No departure data available for this station.")
            return pd.DataFrame()

        # Convert departure data to DataFrame
        df = pd.DataFrame(data["Departure"])
        return df
    except requests.exceptions.RequestException as err:
        st.error(f"Network or HTTP error: {err}")
        return pd.DataFrame()

In [6]:
df_ = fetch_timetable(ext_id)
df_.head(5)

,JourneyDetailRef,JourneyStatus,ProductAtStop,Product,Notes,name,type,stop,stopid,stopExtId,lon,lat,time,date,reachable,direction,directionFlag
0,{'ref': '1|64800|3|1|29012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 8,ST,Angered centrum (Göteborg kn),A=1@O=Angered centrum (Göteborg kn)@X=12049213...,740025606,12.049213,57.796073,09:23:00,2025-01-29,True,Frölunda torg (Göteborg kn),2
1,{'ref': '1|118863|6|1|29012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Buss 401,ST,Angered centrum (Göteborg kn),A=1@O=Angered centrum (Göteborg kn)@X=12049213...,740025606,12.049213,57.796073,09:24:00,2025-01-29,True,Kungälv resecentrum,2
2,{'ref': '1|105977|2|1|29012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 9,ST,Angered centrum (Göteborg kn),A=1@O=Angered centrum (Göteborg kn)@X=12049213...,740025606,12.049213,57.796073,09:26:00,2025-01-29,True,Göteborg Kungssten,2
3,{'ref': '1|27981|7|1|29012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Buss 74,ST,Angered centrum (Göteborg kn),A=1@O=Angered centrum (Göteborg kn)@X=12049213...,740025606,12.049213,57.796073,09:28:00,2025-01-29,True,Partille centrum,1
4,{'ref': '1|30053|1|1|29012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 4,ST,Angered centrum (Göteborg kn),A=1@O=Angered centrum (Göteborg kn)@X=12049213...,740025606,12.049213,57.796073,09:30:00,2025-01-29,True,Mölndals Innerstad,2


In [7]:
# Function to calculate minutes remaining for departures
def calculate_minutes_remaining(df):
    if df.empty:
        return df

    current_time = datetime.now()
    df['departure_datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    df['minutes_remaining'] = (df['departure_datetime'] - current_time).dt.total_seconds() // 60
    
    df = df[df['minutes_remaining'] >= 0]
    
    # Make it User friendly the interface
    df['name'] = df['name'].str.replace(r'^Länstrafik - ', '', regex=True)
    df['stop'] = df['stop'].str.replace(r'\s*\(Göteborg kn\)', '', regex=True)
    df['direction'] = df['direction'].str.replace(r'\s*\(Göteborg kn\)', '', regex=True)

    return df[['name', 'stop', 'direction', 'minutes_remaining']]


In [8]:
df = fetch_timetable(ext_id)  # Run this first
df__ = calculate_minutes_remaining(df)  # Then run this
df__.head(6)


/var/folders/5w/rtw5hcjn25q4yp0062_8_m8h0000gn/T/ipykernel_17104/2819433555.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = df['name'].str.replace(r'^Länstrafik - ', '', regex=True)
/var/folders/5w/rtw5hcjn25q4yp0062_8_m8h0000gn/T/ipykernel_17104/2819433555.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stop'] = df['stop'].str.replace(r'\s*\(Göteborg kn\)', '', regex=True)
/var/folders/5w/rtw5hcjn25q4yp0062_8_m8h0000gn/T/ipykernel_17104/2819433555.py:15: SettingWithCopyWarning: 


,name,stop,direction,minutes_remaining
1,Buss 401,Angered centrum,Kungälv resecentrum,0.0
2,Spårväg 9,Angered centrum,Göteborg Kungssten,2.0
3,Buss 74,Angered centrum,Partille centrum,4.0
4,Spårväg 4,Angered centrum,Mölndals Innerstad,6.0
5,Buss 73,Angered centrum,Peppargatan,6.0
6,Buss 76,Angered centrum,Angered centrum,6.0


In [46]:
import pandas as pd
from backend.connect_to_api import ResRobot
import requests
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta

# Load environment variables
load_dotenv()

API_KEY = os.getenv("API_KEY")

# Initialize ResRobot
timetable = ResRobot()

# Define location
location_name = "Angered Centrum"
ext_id = timetable.get_location_id(location_name)

if isinstance(ext_id, int):  
    ext_id = str(ext_id)  
else:
    raise ValueError(f"No valid extId found for '{location_name}'. Check the stop name and try again.")



In [47]:
import re

def clean_text(text):
    """Removes anything in parentheses and extra prefixes like 'Länstrafik - '"""
    if pd.isna(text):  # Handle NaN values safely
        return ""
    text = re.sub(r"Länstrafik - ", "", text)  # Remove 'Länstrafik - '
    text = re.sub(r"\s*\(.*?\)", "", text)  # Remove text inside parentheses
    return text.strip()

def fetch_clean_timetable_one_hour_ahead(ext_id):
    """Fetches and cleans the timetable, showing departures exactly one hour ahead."""
    
    future_time = (datetime.now() + timedelta(hours=1)).strftime("%H:%M")
    url = f"https://api.resrobot.se/v2.1/departureBoard?id={ext_id}&format=json&accessId={API_KEY}&time={future_time}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if "Departure" not in data:
            return pd.DataFrame()
        
        df = pd.DataFrame(data["Departure"])
        
        # Extract relevant columns
        df_filtered = df[["name", "stop", "direction", "time"]].copy()
        
        # Clean up text
        df_filtered["name"] = df_filtered["name"].apply(clean_text)
        df_filtered["stop"] = df_filtered["stop"].apply(clean_text)
        df_filtered["direction"] = df_filtered["direction"].apply(clean_text)
        
        # Convert 'time' to HH:MM format and rename column
        df_filtered["one_hour_ahead"] = pd.to_datetime(df_filtered["time"]).dt.strftime("%H:%M")
        
        # Drop original 'time' column
        df_filtered.drop(columns=["time"], inplace=True)

        return df_filtered
    
    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")
        return pd.DataFrame()


In [48]:
# Fetch timetable one hour ahead
future_timetable = fetch_clean_timetable_one_hour_ahead(ext_id)

# Display results
future_timetable


/var/folders/5w/rtw5hcjn25q4yp0062_8_m8h0000gn/T/ipykernel_17104/1297872584.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_filtered["one_hour_ahead"] = pd.to_datetime(df_filtered["time"]).dt.strftime("%H:%M")


,name,stop,direction,one_hour_ahead
0,Buss 76,Angered centrum,Angered centrum,17:10
1,Spårväg 9,Angered centrum,Göteborg Kungssten,17:10
2,Buss 75,Angered centrum,Persiljegatan,17:10
3,Buss 71,Angered centrum,Eriksbo Östergärde,17:10
4,Buss 73,Angered centrum,Peppargatan,17:12
...,...,...,...,...
56,Spårväg 4,Angered centrum,Mölndals Innerstad,18:01
57,Spårväg 9,Angered centrum,Göteborg Kungssten,18:05
58,Spårväg 8,Angered centrum,Frölunda torg,18:07
59,Buss 74,Angered centrum,Partille centrum,18:08


In [32]:
# Function to filter departures after one hour ahead from current time
def one_hour_ahead(df):
    if df.empty:
        return df

    # Get current time and add 60 minutes (1 hour)
    current_time = datetime.now()
    one_hour_ahead_time = current_time + pd.Timedelta(hours=1)

    # Print current time and one hour ahead for debugging
    print("Current time:", current_time)
    print("One hour ahead:", one_hour_ahead_time)

    # Convert 'date' and 'time' columns to datetime
    df['departure_datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], errors='coerce')

    # Print the departure_datetime column for debugging
    print("Departure datetime column (before filtering):")
    print(df[['date', 'time', 'departure_datetime']].head(10))  # Print the first 10 rows for better insight

    # Debug: Check if there are any departures after one hour ahead
    print("Departures after 1 hour ahead:")
    print(df[df['departure_datetime'] > one_hour_ahead_time][['name', 'stop', 'direction', 'departure_datetime']])

    # Filter departures that are strictly after one hour ahead from current time
    df_filtered = df[df['departure_datetime'] >= one_hour_ahead_time]  # Try '>=', not just '>'

    # Print the filtered DataFrame for debugging
    print("Filtered DataFrame (after applying filter):")
    print(df_filtered[['name', 'stop', 'direction', 'departure_datetime']].head(10))  # Show the filtered data

    # Clean up column values for better readability
    df_filtered.loc[:, 'name'] = df_filtered['name'].str.replace(r'^Länstrafik - ', '', regex=True)
    df_filtered.loc[:, 'stop'] = df_filtered['stop'].str.replace(r'\s*\(Göteborg kn\)', '', regex=True)
    df_filtered.loc[:, 'direction'] = df_filtered['direction'].str.replace(r'\s*\(Göteborg kn\)', '', regex=True)

    # Extract only the time (no date)
    df_filtered['departure_time'] = df_filtered['departure_datetime'].dt.strftime('%H:%M')

    # Drop the original datetime column
    df_filtered = df_filtered.drop(columns=['departure_datetime'])

    return df_filtered[['name', 'stop', 'direction', 'departure_time']]

# Fetch and process timetable data
df = fetch_timetable(ext_id)
df_processed = one_hour_ahead(df)

# Display the filtered data for departures after 1 hour ahead from the current time
print(df_processed.head(6))



Current time: 2025-01-29 13:57:40.471988
One hour ahead: 2025-01-29 14:57:40.471988
Departure datetime column (before filtering):
         date      time  departure_datetime
0  2025-01-29  13:58:00 2025-01-29 13:58:00
1  2025-01-29  13:58:00 2025-01-29 13:58:00
2  2025-01-29  14:00:00 2025-01-29 14:00:00
3  2025-01-29  14:01:00 2025-01-29 14:01:00
4  2025-01-29  14:01:00 2025-01-29 14:01:00
5  2025-01-29  14:04:00 2025-01-29 14:04:00
6  2025-01-29  14:05:00 2025-01-29 14:05:00
7  2025-01-29  14:06:00 2025-01-29 14:06:00
8  2025-01-29  14:06:00 2025-01-29 14:06:00
9  2025-01-29  14:06:00 2025-01-29 14:06:00
Departures after 1 hour ahead:
Empty DataFrame
Columns: [name, stop, direction, departure_datetime]
Index: []
Filtered DataFrame (after applying filter):
Empty DataFrame
Columns: [name, stop, direction, departure_datetime]
Index: []
Empty DataFrame
Columns: [name, stop, direction, departure_time]
Index: []
